# Data Preparation and Feature Engineering

This notebook documents the process of cleaning the raw metadata and generating species-level features for the MicroArk project.

In [1]:
import pandas as pd
import os
from pathlib import Path

# Define paths relative to the project root
# Assuming this notebook is in src/notebooks/
project_root = Path("../../").resolve()
input_file = project_root / "data" / "metalog_raw.csv"
output_file = project_root / "data" / "species_features.csv"

print(f"Project Root: {project_root}")
print(f"Input File: {input_file}")

Project Root: G:\Projects\MicroArk
Input File: G:\Projects\MicroArk\data\metalog_raw.csv


## 1. Load Data
Load the raw metadata CSV downloaded from Metalog.

In [2]:
if not input_file.exists():
    print(f"Error: {input_file} not found.")
else:
    df = pd.read_csv(input_file, low_memory=False)
    print(f"Loaded DataFrame with shape: {df.shape}")
    display(df.head())

Loaded DataFrame with shape: (11218, 58)


,study_code,sample_alias,spire_sample_name,longitude,latitude,geographic_location,location_resolution,collection_date,collection_date_end,last_change,...,sample_collection_timepoint,antibiotic,age_range,medication_full,study_accession,elevation_meters,sample_description,host,depth_meters,sample_title
0,Pan_2018_rat,Pan_2018_rat.sample_110,SAMEA104346255,113.26,31.99,China,country,2016-01-01,2016-12-31,2025-08-12 09:33:32.131673,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stool sample from Rattus norvegicus BN/Sprague...
1,PRJEB12987_chicken,PRJEB12987_chicken.XT1,SAMEA3891233,12.15,42.73,Italy,country,2015-05-12,NaN,2025-08-12 09:30:11.748390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Group1
2,PRJEB12987_chicken,PRJEB12987_chicken.XT19,SAMEA3891869,12.15,42.73,Italy,country,2015-05-05,NaN,2025-08-12 09:30:11.748390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Group C
3,PRJEB12987_chicken,PRJEB12987_chicken.XT28,SAMEA3891878,12.15,42.73,Italy,country,2015-05-05,NaN,2025-08-12 09:30:11.748390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Group C+FT
4,PRJEB12987_chicken,PRJEB12987_chicken.XT31,SAMEA3891881,12.15,42.73,Italy,country,2015-05-05,NaN,2025-08-12 09:30:11.748390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Group C+FT


## 2. Preprocessing & Column Mapping

We need to map specific columns to our target features:
- `host_species` <--- `host_tax_id` (Taxonomic ID is more reliable)
- `country` <--- `location` (Extracted from string)
- `collection_year` <--- `collection_date` (Extracted year)

In [3]:
# Map host_species
if 'host_tax_id' in df.columns:
    df['host_species'] = df['host_tax_id']
else:
    print("Error: 'host_tax_id' column missing.")

# Map country
if 'location' in df.columns:
    df['country'] = df['location'].astype(str).apply(lambda x: x.split(':')[0].strip() if pd.notnull(x) else "Unknown")
else:
    df['country'] = "Unknown"

# Map collection_year
if 'collection_date' in df.columns:
    df['collection_year'] = pd.to_datetime(df['collection_date'], errors='coerce').dt.year
else:
    df['collection_year'] = 0

# Ensure coordinates
for col in ['latitude', 'longitude']:
    if col not in df.columns:
        df[col] = 0

print("Columns mapped.")

Columns mapped.


## 3. Data Cleaning

1. Drop rows where `host_species` is NaN.
2. Filter out species that have fewer than 5 samples.

In [4]:
before_count = len(df)
df = df.dropna(subset=["host_species"])
print(f"Dropped {before_count - len(df)} rows with missing species info.")

# Filter < 5 samples
species_counts = df["host_species"].value_counts()
valid_species = species_counts[species_counts >= 5].index
df_filtered = df[df["host_species"].isin(valid_species)].copy()

print(f"Retained {len(valid_species)} species with >= 5 samples.")
print(f"Final sample count: {len(df_filtered)}")

Dropped 0 rows with missing species info.
Retained 116 species with >= 5 samples.
Final sample count: 10770


## 4. Feature Creation

Group by `host_species` and calculate:
- `num_samples`: Total count
- `num_countries`: Unique countries
- `year_span`: Max year - Min year
- `lat_variance`: Variance of latitude
- `long_variance`: Variance of longitude

In [5]:
grouped = df_filtered.groupby("host_species")

features = pd.DataFrame({
    "num_samples": grouped.size(),
    "num_countries": grouped["country"].nunique(),
    "year_span": grouped["collection_year"].max() - grouped["collection_year"].min(),
    "lat_variance": grouped["latitude"].var(),
    "long_variance": grouped["longitude"].var()
})

features = features.fillna(0)
features.reset_index(inplace=True)

display(features.head())

,host_species,num_samples,num_countries,year_span,lat_variance,long_variance
0,6563,60,1,0,0.0,0.0
1,6565,16,1,0,0.0,0.0
2,6689,40,1,0,0.0,0.0
3,6763,24,1,0,0.0,0.0
4,6941,16,1,0,0.0,0.0


## 5. Save Results
Save the generated features to a CSV file.

In [6]:
features.to_csv(output_file, index=False)
print(f"Saved features to {output_file}")

Saved features to G:\Projects\MicroArk\data\species_features.csv
